##掛載 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##加入專案路徑

In [ ]:
import os
import sys

# 您的專案在 Google Drive 中的路徑
PROJECT_PATH = '/content/drive/MyDrive/projects/LINE BOT'

# 您要檢查的檔案名稱
FILE_CHECK_NAME = 'v_2_keyword_extractor.py'

# 組合完整的檔案路徑
FILE_CHECK_PATH = os.path.join(PROJECT_PATH, FILE_CHECK_NAME)

# --- 診斷邏輯 ---
if os.path.exists(FILE_CHECK_PATH):
    print(f"✅ 診斷成功：文件 '{FILE_CHECK_NAME}' 存在於 {PROJECT_PATH}。")

    # 僅在路徑尚未加入時才加入，以避免重複
    if PROJECT_PATH not in sys.path:
        sys.path.append(PROJECT_PATH)
        print(f"✅ 已將路徑加入系統搜尋清單。")
else:
    # 如果文件不存在，立即停止並報告錯誤
    print(f"❌ 診斷失敗：文件 '{FILE_CHECK_NAME}' 不存在於 {PROJECT_PATH}。")
    print("請仔細檢查 Google Drive 上的資料夾名稱是否有隱藏的空格或符號。")
    raise FileNotFoundError("路徑或檔案名稱錯誤，無法繼續。")
# --- 診斷結束 ---

##1) 安裝套件（指定版本範圍＋用途註解）

In [ ]:
# Colab Cell (安裝套件)

# 1. 安裝 LINE Bot SDK v3
!pip install "line-bot-sdk>=3.0.0,<4.0.0"

# 2. 安裝 Flask
!pip install "Flask>=3.0.0,<4.0.0"

# 3. 安裝 pyngrok
!pip install "pyngrok>=7.0.0,<8.0.0"

# 4. Jieba: 中文分詞庫 (用於 TF-IDF 提取)
!pip install "jieba>=0.42"

##2) 開 ngrok（先執行這一格拿到公開網址）

In [ ]:
from pyngrok import ngrok

# 換成你自己的 ngrok authtoken
NGROK_AUTHTOKEN = "你的密碼"   # 例：2pEWt4...（不要公開在 GitHub）
PORT = 5000

# 設定金鑰
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 關閉舊隧道（避免重複）
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

# 建立新隧道，取得公開 HTTPS 網址
public_url = ngrok.connect(PORT, "http").public_url

print("👉 外網網址：", public_url)
print("Webhook URL：", public_url + "/callback")

##3) LINE Bot 程式

In [ ]:
#從Line bot sdk pypl網站找到程式碼貼上
#https://pypi.org/project/line-bot-sdk/
import os
import sys
import re
import json # 🌟 新增：用於處理關鍵詞的 JSON 格式 🌟
from flask import Flask, request, abort

from v1_sqlite_dataseat import (
    init_db,
    insert_log,
    close_db,
    get_user_log_count,
    # 🌟 修正：導入所有需要的 CRUD 函式 🌟
    get_latest_log,
    get_first_log,
    get_nth_log,
    delete_all_logs,
    delete_log_by_id,
    get_all_logs_summary,
     # 🌟 V4 新增導入 🌟
    get_weekly_logs  # <--- 請務必新增這一行
)

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

# 1. 導入您的資料庫函式
from v1_sqlite_dataseat import (
    # ... (原有 DB 函式)
    get_all_logs_summary
)

# 🌟 V2 關鍵字提取：導入新的分析函式 🌟
from v_2_keyword_extractor import extract_keywords_tfidf
# -----------------------------------------------


app = Flask(__name__)

#新增模擬 AI API 函式
def call_ai_summary_api(all_logs_text):
    """
    [模擬] 將彙整的日記內容傳給 AI 模型 API 以獲取總結。
    在正式實施時，這裡將替換為真實的 HTTP 請求 (如 requests.post)。
    """
    if not all_logs_text.strip():
        return "無法生成總結，因為過去一週沒有任何日記內容。"

    # 計算總字數 (作為一個簡單的「分析」結果)
    word_count = len(all_logs_text)

    # 簡單模擬 AI 模型的總結結果
    summary = (
        f"✅ AI 彙整結果：\n"
        f"您在過去一週共記錄了 {word_count} 個字。\n"
        f"您的主要的情感波動在計畫的完成與對自己的要求過於嚴格。\n"
        f"建議您多放鬆，並安排休息時間以平衡身心。"
    )
    return summary

# 憑證設定：請替換成您的 LINE Bot 憑證
CHANNEL_ACCESS_TOKEN = os.environ.get('LINE_ACCESS_TOKEN', 'YOUR_LINE_ACCESS_TOKEN')
CHANNEL_SECRET = os.environ.get('LINE_CHANNEL_SECRET', 'YOUR_LINE_CHANNEL_SECRET')

configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 🌟 數據庫優化：註冊資料庫關閉函式 🌟
# 確保每個請求結束後都會關閉連線，釋放資源
app.teardown_appcontext(close_db)

# 🌟 修正點：定義 CONFIRM_DELETE_MSG 🌟
CONFIRM_DELETE_MSG = "請輸入 **確定刪除** 以完成清空操作，否則請忽略本訊息。"

DELETE_ID_PATTERN = re.compile(r'刪除\s*ID\s*:\s*(\d+)', re.IGNORECASE)
DELETE_ALL_PATTERN = re.compile(r'刪除所有')
QUERY_PATTERN = re.compile(r'第(\d+)篇')


@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_id = event.source.user_id
    user_input = event.message.text.strip()
    reply_text = None
    # ----------------------------------------------------
    # A. 處理「刪除」指令 (已移除海象運算子)
    # ----------------------------------------------------
    # 1. 處理「確認刪除」回覆
    if user_input == "確定刪除":

        deleted_count = delete_all_logs(user_id)
        if deleted_count > 0:
            reply_text = (
                f"🗑️ 警告：已成功刪除您共 {deleted_count} 篇日記！您的紀錄已清空。\n"
                f"現在您可以重新開始寫日記了！"
            )
        else:
            reply_text = "您的日記庫中沒有任何紀錄可以刪除，不需要進行清空操作。"
    # 2. 處理「刪除 ID」指令 (修正：移除海象運算子)
    match = DELETE_ID_PATTERN.match(user_input.lower())
    if reply_text is None and match: # 使用 if match 來處理賦值
        log_id_to_delete = int(match.group(1))

    # 🌟 修正：新增 try...except 區塊來捕捉潛在的資料庫錯誤 🌟
        try:
        # 嘗試執行刪除
            if delete_log_by_id(user_id, log_id_to_delete): # 修正：delete_log_by_id 需要 user_id
                total_logs = get_user_log_count(user_id)
                reply_text = (
                   f"🗑️ 已成功刪除唯一 ID 為 {log_id_to_delete} 的紀錄！\n"
                  f"目前您還剩下 {total_logs} 篇日記。"
                 )
            else:
            # 刪除失敗或找不到記錄
                reply_text = f"找不到 ID 為 {log_id_to_delete} 的紀錄可供刪除，或該紀錄不屬於您。"

        except Exception as e:
        # 捕捉到錯誤時，設定錯誤回覆，並將錯誤訊息印出來用於診斷
            print(f"❌ 資料庫錯誤發生於刪除 ID {log_id_to_delete}: {e}")
            reply_text = "⚠️ **抱歉，刪除過程中發生系統錯誤，請檢查伺服器日誌。**"

# 這裡不需要 else，因為 reply_text 在 try/except 中已經設定。


    # 3. 處理「請求刪除所有」指令 (觸發二次確認)
    elif reply_text is None and DELETE_ALL_PATTERN.match(user_input.lower()):
        total_logs = get_user_log_count(user_id)
        if total_logs == 0:
            reply_text = "您的日記庫中沒有任何紀錄可以刪除。"
        else:
            # 🌟 關鍵：要求二次確認 🌟
            reply_text = (
                f"⚠️ **您確定要刪除所有 {total_logs} 篇日記嗎？**\n"
                f"此操作不可復原！\n\n"
                f"{CONFIRM_DELETE_MSG}" # 確保 CONFIRM_DELETE_MSG 已定義
            )
# ----------------------------------------------------
    # B. 處理「清單/預覽所有」指令 (ELIF)
    # ----------------------------------------------------
    # 這裡使用 elif 確保只有在 A 區塊未命中時才執行
    elif user_input.lower() in ['/list', '清單', '預覽所有']:
        summaries = get_all_logs_summary(user_id)
        total_logs = len(summaries)

        if total_logs > 0:
            list_items = []
            # i 是從 0 開始的索引
            for i, (log_id, date_part, summary) in enumerate(summaries):
                # 顯示格式: [編號] (日期) ID: XXX | 摘要內容
                list_items.append(
                    f"[{i+1}] ({date_part}) ID:{log_id} | {summary}"
                )

            reply_text = (
                f"📝 您的日記清單 (共 {total_logs} 篇)：\n"
                f"--------------------------------------\n"
                + "\n".join(list_items) +
                f"\n--------------------------------------\n"
                f"可輸入『刪除 ID: XXX』或『第 N 篇』進行操作。\n"
                f"如需全部刪除，請輸入『刪除所有』。"
            )
        else:
            reply_text = "您的日記庫中沒有任何紀錄喔！"

    # ----------------------------------------------------
    # 🌟 V3 處理「每週總結」指令 (ELIF) 🌟
    # ----------------------------------------------------
    elif user_input.lower() in ['/summary', '每週總結', '總結']:

        weekly_logs = get_weekly_logs(user_id, days=7)

        if not weekly_logs:
            reply_text = "過去七天沒有任何日記紀錄，無法為您生成每週總結。"
        else:
            all_logs_text = "\n---\n".join(weekly_logs)
            summary_result = call_ai_summary_api(all_logs_text)

            reply_text = (
                f"📅 **【您的每週 (7 天) 總結】** 📅\n"
                f"您共寫了 **{len(weekly_logs)}** 篇日記。\n"
                f"----------------------------\n"
                f"{summary_result}"
            )

   # ----------------------------------------------------
    # C. 處理「查詢單篇」指令 (ELIF) - 修正：移除海象運算子
    # ----------------------------------------------------
    elif user_input.lower() in ['/last', '最後一篇', '查詢最新'] or \
         user_input.lower() in ['/first', '第一篇', '查詢最早'] or \
         QUERY_PATTERN.match(user_input): # 這裡只做判斷

        log = None
        title = None
        n = None
        total_logs = get_user_log_count(user_id)

        # 執行查詢，並設定 log, title, n
        if user_input.lower() in ['/last', '最後一篇', '查詢最新']:
            log = get_latest_log(user_id)
            title = "最新"
        elif user_input.lower() in ['/first', '第一篇', '查詢最早']:
            log = get_first_log(user_id)
            title = "第一篇"

        # 🌟 修正：將海象運算子替換為標準 if 結構 🌟
        match = QUERY_PATTERN.match(user_input)
        if match:
            n = int(match.group(1))
            log = get_nth_log(user_id, n)
            title = f"第 {n} 篇"

        # 根據查詢結果組合回覆
        if log:
            log_id, log_text = log
            reply_text = (
                f"📜 您寫下的**{title}**紀錄是：\n"
                f"👉 唯一 ID: {log_id} 👈\n"
                f"--------------------\n"
                f"「{log_text}」\n"
                f"--------------------\n"
                f"如需刪除此篇日記，請輸入『刪除 ID: {log_id}』進行操作，否則忽略本訊息。"
            )
        elif total_logs == 0:
            reply_text = "您還沒有寫過任何紀錄喔！"
        elif n and n > total_logs:
             reply_text = f"目前您只寫了 {total_logs} 篇紀錄，無法查詢第 {n} 篇。"


    # ----------------------------------------------------
    # D. 處理「寫日記」指令 (ELIF reply_text is None)
    # ----------------------------------------------------
    # 這是最後的指令！確保只有在 reply_text 仍為 None 時才執行
    elif reply_text is None:

        extracted_keywords_json = extract_keywords_tfidf(user_input)

        # 儲存日記
        insert_log(
            user_id=user_id,
            raw_text=user_input,
            keywords=extracted_keywords_json
        )
        total_logs = get_user_log_count(user_id)

        # 組合回覆
        try:
            keywords_display = json.loads(extracted_keywords_json)
        except json.JSONDecodeError:
            keywords_display = ["提取失敗或格式錯誤"]

        reply_text = (
            f"📝 您的紀錄已成功收錄！\n"
            f"「這是您寫下的第 {total_logs} 篇紀錄。」\n"
            f"🔑 系統提取的關鍵詞：{keywords_display} \n"
            f" ❤️我們會溫柔地為您保管這份珍貴的回憶。\n"
            f"👉輸入『清單』即可預覽所有紀錄清單！\n"
            f"👉輸入『每週總結』即可預覽本週紀錄"
        )


    # ----------------------------------------------------
    # E. 回覆使用者
    # ----------------------------------------------------
    if reply_text:
        # 請確保這裡使用您環境中正確的 Line Bot API 初始化和呼叫方式
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            try:
                line_bot_api.reply_message(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                print(f"回覆使用者時發生錯誤: {e}")

if __name__ == "__main__":
    # 應用程式啟動前，先初始化資料庫
    init_db()

    # 運行 Flask 服務
    app.run(port=5000)
